In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report,confusion_matrix
import ipywidgets as widgets
import io
from PIL import Image
from IPython.display import display,clear_output
from warnings import filterwarnings

In [2]:
dir = os.path.dirname
path = dir(os.getcwd())
print(path)
sys.path.append(path)

d:\COSAS\BOOTCAMP\MLProyect


In [3]:
labels = ['glioma_tumor','no_tumor','meningioma_tumor','pituitary_tumor']

In [4]:
X_train = []
y_train = []
pathlist = []
dicc = []
image_size = 150
for i in labels:
    folderPath = dir(os.getcwd()) + os.sep + "data"+ os.sep +'Training' + os.sep + i
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_train.append(i)
        pathlist.append(folderPath + os.sep + j)
        dicc.append({"Image":img, "Label":i, "Fullpath":os.path.join(folderPath,j)})

X_train = np.array(X_train)
y_train = np.array(y_train)
pathlist = np.array(pathlist)

df_train = pd.DataFrame(dicc)
df_train.head()

100%|██████████| 827/827 [00:04<00:00, 203.32it/s]


,Image,Label,Fullpath
0,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",glioma_tumor,d:\COSAS\BOOTCAMP\MLProyect\data\Training\glio...
1,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",glioma_tumor,d:\COSAS\BOOTCAMP\MLProyect\data\Training\glio...
2,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",glioma_tumor,d:\COSAS\BOOTCAMP\MLProyect\data\Training\glio...
3,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",glioma_tumor,d:\COSAS\BOOTCAMP\MLProyect\data\Training\glio...
4,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",glioma_tumor,d:\COSAS\BOOTCAMP\MLProyect\data\Training\glio...


In [5]:
#Creating test data
X_test = []
y_test = []
pathlist_test = []
dicc_test = []
image_size = 150
for i in labels:
    folderpath_test = dir(os.getcwd()) + os.sep + 'data'+ os.sep +'Testing'+ os.sep + i
    for j in tqdm(os.listdir(folderpath_test)):
        img = cv2.imread(os.path.join(folderpath_test,j))
        img = cv2.resize(img,(image_size,image_size))
        X_test.append(img)
        y_test.append(i)
        dicc_test.append({"Image":img, "Label":i, "Fullpath":os.path.join(folderpath_test,j)})
X_test = np.array(X_test)
y_test = np.array(y_test)
pathlist_test = np.array(pathlist_test)

df_test = pd.DataFrame(dicc_test)
df_test.head()


100%|██████████| 74/74 [00:00<00:00, 104.94it/s]


,Image,Label,Fullpath
0,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",glioma_tumor,d:\COSAS\BOOTCAMP\MLProyect\data\Testing\gliom...
1,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",glioma_tumor,d:\COSAS\BOOTCAMP\MLProyect\data\Testing\gliom...
2,"[[[7, 7, 7], [7, 7, 7], [7, 7, 7], [6, 6, 6], ...",glioma_tumor,d:\COSAS\BOOTCAMP\MLProyect\data\Testing\gliom...
3,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",glioma_tumor,d:\COSAS\BOOTCAMP\MLProyect\data\Testing\gliom...
4,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",glioma_tumor,d:\COSAS\BOOTCAMP\MLProyect\data\Testing\gliom...


In [9]:
df_photos = df_train.append(df_test)

In [ ]:
df_photos

In [10]:
df_photos.to_csv('../data/df_photos.csv', index=False)

In [ ]:
df_photos.to_json()

In [39]:
X_train.shape

(2870, 150, 150, 3)

In [40]:
y_train.shape

(2870,)

In [41]:
X_test.shape

(394, 150, 150, 3)

In [78]:
X_train = np.array(X_train).reshape(1,-1)
y_train = np.array(y_train).reshape(1,-1)
pathlist = np.array(pathlist).reshape(1,-1)

In [68]:
X_train

array([[0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [69]:
y_train

array([['glioma_tumor', 'glioma_tumor', 'glioma_tumor', ...,
        'pituitary_tumor', 'pituitary_tumor', 'pituitary_tumor']],
      dtype='<U16')

In [44]:
y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)


y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

In [27]:
X_train, y_train = shuffle(X_train,y_train, random_state=153)

In [13]:
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True)

datagen.fit(X_train)
X_train.shape

(2870, 150, 150, 3)

In [27]:
X_train,X_test,y_train,y_test = train_test_split(X_train,y_train, test_size=0.2,random_state=153)

In [45]:
effnet = EfficientNetB0(weights='imagenet',include_top=False,input_shape=(image_size,image_size,3))

In [46]:
model = effnet.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(rate=0.5)(model)
model = tf.keras.layers.Dense(4,activation='softmax')(model)
model = tf.keras.models.Model(inputs=effnet.input, outputs = model)

In [47]:
model.compile(loss='categorical_crossentropy',optimizer = 'Adam', metrics= ['accuracy'])

In [49]:
history = model.fit(X_train,y_train,validation_split=0.1, epochs =5, verbose=1, batch_size=32)

Epoch 1/5
81/81 [==============================] - 239s 3s/step - loss: 0.2212 - accuracy: 0.9199 - val_loss: 1.8457 - val_accuracy: 0.5854
Epoch 2/5
81/81 [==============================] - 229s 3s/step - loss: 0.1125 - accuracy: 0.9593 - val_loss: 0.5741 - val_accuracy: 0.8362
Epoch 3/5
81/81 [==============================] - 239s 3s/step - loss: 0.0746 - accuracy: 0.9741 - val_loss: 0.7051 - val_accuracy: 0.8049
Epoch 4/5
81/81 [==============================] - 228s 3s/step - loss: 0.0621 - accuracy: 0.9783 - val_loss: 0.4743 - val_accuracy: 0.8711
Epoch 5/5
81/81 [==============================] - 224s 3s/step - loss: 0.1007 - accuracy: 0.9698 - val_loss: 0.0628 - val_accuracy: 0.9895


In [50]:
pred = model.predict(X_test)
pred = np.argmax(pred,axis=1)
y_test_new = np.argmax(y_test,axis=1)

In [51]:
print(classification_report(y_test_new,pred))

              precision    recall  f1-score   support

           0       0.86      0.25      0.39       100
           1       0.80      0.97      0.88       105
           2       0.75      0.87      0.81       115
           3       0.65      0.92      0.76        74

    accuracy                           0.75       394
   macro avg       0.77      0.75      0.71       394
weighted avg       0.77      0.75      0.71       394



0 = 'glioma_tumor'
1 = 'no_tumor'
2 = 'meningioma_tumor'
3 = 'pituitary_tumor'

In [52]:
model.save('../models/model_2.h5')

C:\Users\marin\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
#This worked here but not with functions, i will leave it this way